In [1]:

from collections import Counter

import numpy as np


import torch

from datasets import (
    
    load_from_disk,
)
from transformers import (
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)


import pickle

from sklearn.metrics import  classification_report
from transformers import BertTokenizerFast, BertForTokenClassification

from evaluate import load as load_metric

/scistor/guest/lre040/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "bert-base-uncased"

In [3]:
conll_main = load_from_disk("./splits/conll_main")

ontonotes_main = load_from_disk("./splits/ontonotes_main")

ontonotes_nw_encoded = load_from_disk("./splits/ontonotes_newswire")
ontonotes_rest_encoded = load_from_disk("./splits/ontonotes_rest")

# Load GPU

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Tokenisation & Alignment

In [5]:
ontonotes_id_to_label = {
    0: "O", 1: "B-CARDINAL", 2: "B-DATE", 3: "I-DATE", 4: "B-PERSON", 5: "I-PERSON",
    6: "B-NORP", 7: "B-GPE", 8: "I-GPE", 9: "B-LAW", 10: "I-LAW", 11: "B-ORG", 12: "I-ORG",
    13: "B-PERCENT", 14: "I-PERCENT", 15: "B-ORDINAL", 16: "B-MONEY", 17: "I-MONEY",
    18: "B-WORK_OF_ART", 19: "I-WORK_OF_ART", 20: "B-FAC", 21: "B-TIME", 22: "I-CARDINAL",
    23: "B-LOC", 24: "B-QUANTITY", 25: "I-QUANTITY", 26: "I-NORP", 27: "I-LOC",
    28: "B-PRODUCT", 29: "I-TIME", 30: "B-EVENT", 31: "I-EVENT", 32: "I-FAC",
    33: "B-LANGUAGE", 34: "I-PRODUCT", 35: "I-ORDINAL", 36: "I-LANGUAGE"
}

conll_label_to_id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3,
                     'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
id2label = {v: k for k, v in conll_label_to_id.items()}

ontonotes_to_conll_entity = {
    "PERSON": "PER", "ORG": "ORG", "GPE": "LOC", "LOC": "LOC",
    "NORP": "MISC", "FAC": "MISC", "EVENT": "MISC", "WORK_OF_ART": "MISC",
    "LAW": "MISC", "PRODUCT": "MISC", "LANGUAGE": "MISC",
    "DATE": None, "TIME": None, "PERCENT": None, "MONEY": None,
    "QUANTITY": None, "ORDINAL": None, "CARDINAL": None
}

In [6]:
def process_data(data_list):

    def process_single(data):
        word_ids = data['word_ids']
        predictions = data['predictions']
        gold = data['gold']
        tokenized_tokens = data['tokens']

        word_ids = [a for a in word_ids if a is not None]

        processed_predictions = []
        processed_gold = []

        current_word_id = None
        current_predictions = []
        current_gold = []

        for idx, word_id in enumerate(word_ids):
            if word_id != current_word_id:
                if current_predictions:
                    processed_predictions.append(
                        Counter(current_predictions).most_common(1)[0][0])
                    processed_gold.append(
                        Counter(current_gold).most_common(1)[0][0])

                current_word_id = word_id
                current_predictions = [predictions[idx]]
                current_gold = [gold[idx]]
            else:
                current_predictions.append(predictions[idx])
                current_gold.append(gold[idx])

        if current_predictions:
            processed_predictions.append(
                Counter(current_predictions).most_common(1)[0][0])
            processed_gold.append(
                Counter(current_gold).most_common(1)[0][0])

        return processed_predictions, processed_gold

    processed_predictions_list = []
    processed_gold_list = []

    for data in data_list:
        processed_predictions, processed_gold = process_single(data)
        processed_predictions_list.append(processed_predictions)
        processed_gold_list.append(processed_gold)

    return processed_predictions_list, processed_gold_list


def evaluate_predictions(p, test_data):
    predictions, labels, _ = p

    pred_indices = [np.argmax(p, axis=-1) for p in predictions]
    label_indices = labels

    pred_tags = [[id2label[p] for p, l in zip(p_seq, l_seq) if l != -100]
                 for p_seq, l_seq in zip(pred_indices, label_indices)]
    gold_tags = [[id2label[l] for l in l_seq if l != -100]
                 for l_seq in label_indices]

    def add_preds(example, idx):
        length = len(example['word_ids'])
        example['predictions'] = pred_tags[idx][:length]
        example['gold'] = gold_tags[idx][:length]
        return example

    test_data = test_data.map(add_preds, with_indices=True)

    length = len(test_data['predictions'][0])

    pred, gold = process_data(test_data)

    flat_pred = [label for seq in pred for label in seq]
    flat_gold = [label for seq in gold for label in seq]

    print(classification_report(flat_gold, flat_pred, zero_division=0))

    return (flat_pred, flat_gold)

# Model

In [7]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)

label_list = ['O', 'B-PER', 'I-PER', 'B-ORG',
              'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=True,
        return_special_tokens_mask=True,
        return_offsets_mapping=True,
    )
    all_word_ids = []
    all_labels = []
    for i, labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        all_word_ids.append(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(labels[word_idx])
            previous_word_idx = word_idx
        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    tokenized_inputs["word_ids"] = all_word_ids
    return tokenized_inputs

In [8]:
conll_main = conll_main.map(tokenize_and_align_labels, batched=True)
ontonotes_main = ontonotes_main.map(tokenize_and_align_labels, batched=True)
onto_news = ontonotes_nw_encoded.map(tokenize_and_align_labels, batched=True)
onto_rest = ontonotes_rest_encoded.map(tokenize_and_align_labels, batched=True)

# Deciding params

In [9]:
train_data = conll_main
test_data = onto_news

In [10]:
train_data_name = 'news'

In [11]:
mod = BertForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_list))

metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)


data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir=f"./output/{model_name}",
    eval_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none",
    fp16=True,
    save_strategy="no",
)

trainer = Trainer(
    model=mod,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/scratch/1063416/ipykernel_1294453/1248433719.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

Step,Training Loss
500,0.223600
1000,0.080200
1500,0.050400
2000,0.045300
2500,0.029900
3000,0.027500


TrainOutput(global_step=3114, training_loss=0.07433305386939566, metrics={'train_runtime': 394.5068, 'train_samples_per_second': 126.196, 'train_steps_per_second': 7.893, 'total_flos': 3729913475126796.0, 'train_loss': 0.07433305386939566, 'epoch': 3.0})

In [13]:
trainer.save_model(f"./saved_model/{model_name}_{train_data_name}")

In [14]:
predictions = trainer.predict(test_data)

In [15]:
with open(f'./results/{model_name}_{train_data_name}_results.pkl', 'wb') as f:
    pickle.dump(predictions, f)

In [16]:
x, y = evaluate_predictions(predictions, test_data)

Map:   0%|          | 0/42753 [00:00<?, ? examples/s]

Map:   2%|▏         | 667/42753 [00:00<00:06, 6582.85 examples/s]

Map:   3%|▎         | 1348/42753 [00:00<00:08, 5146.49 examples/s]

Map:   5%|▍         | 2000/42753 [00:00<00:08, 4738.80 examples/s]

Map:   6%|▋         | 2709/42753 [00:00<00:07, 5483.48 examples/s]

...

Map:  97%|█████████▋| 41437/42753 [00:08<00:00, 4643.99 examples/s]

Map:  98%|█████████▊| 42000/42753 [00:08<00:00, 4724.30 examples/s]

Map: 100%|█████████▉| 42628/42753 [00:08<00:00, 5086.35 examples/s]

Map: 100%|██████████| 42753/42753 [00:09<00:00, 4423.21 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.82      0.90      0.86     16435
      B-MISC       0.50      0.68      0.58      7455
       B-ORG       0.74      0.73      0.73     22446
       B-PER       0.89      0.88      0.88     12641
       I-LOC       0.68      0.60      0.64      5154
      I-MISC       0.52      0.27      0.36      5579
       I-ORG       0.89      0.72      0.80     30834
       I-PER       0.84      0.92      0.88      7761
           O       0.99      0.99      0.99    965876

    accuracy                           0.97   1074181
   macro avg       0.76      0.74      0.75   1074181
weighted avg       0.97      0.97      0.97   1074181



In [17]:
predictions = trainer.predict(onto_rest)
x, y = evaluate_predictions(predictions, onto_rest)

Map:   0%|          | 0/100516 [00:00<?, ? examples/s]

Map:   1%|          | 760/100516 [00:00<00:13, 7493.37 examples/s]

Map:   2%|▏         | 1967/100516 [00:00<00:14, 6894.92 examples/s]

Map:   3%|▎         | 2911/100516 [00:00<00:14, 6614.09 examples/s]

Map:   4%|▍         | 3826/100516 [00:00<00:15, 6312.23 examples/s]

...

Map:  98%|█████████▊| 99000/100516 [00:18<00:00, 5670.19 examples/s]

Map:  99%|█████████▉| 99758/100516 [00:18<00:00, 6116.40 examples/s]

Map: 100%|█████████▉| 100397/100516 [00:18<00:00, 5693.67 examples/s]

Map: 100%|██████████| 100516/100516 [00:20<00:00, 5002.94 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.58      0.88      0.69     14389
      B-MISC       0.50      0.70      0.58     10792
       B-ORG       0.39      0.55      0.46      7517
       B-PER       0.43      0.90      0.58     14691
       I-LOC       0.53      0.59      0.56      4350
      I-MISC       0.48      0.22      0.31      6920
       I-ORG       0.68      0.51      0.58      9826
       I-PER       0.84      0.85      0.84     11300
           O       0.99      0.98      0.98   1568935

    accuracy                           0.96   1648720
   macro avg       0.60      0.69      0.62   1648720
weighted avg       0.97      0.96      0.97   1648720

